In [7]:
import neilpy
import pandas as pd
import numpy as np
from scipy import stats
import geopandas
from skimage.graph import MCP_Geometric, MCP
from skimage import graph
from shapely.geometry import LineString

In [12]:
# A pile of LCP Functions.  This will get moved.

# From Tobler. 1993. THREE PRESENTATIONS ON GEOGRAPHICAL ANALYSIS AND MODELING
# Returns velocity in km/hr.  S is supplied in DEGREES.
def cost_tobler_hiking_function(S,symmetric=True):

    # Convert to dz/dx
    S = np.tan(np.deg2rad(S))
    
    V = 6 * np.exp(-3.5 * np.abs(S + .05))
    
    if symmetric:
        V2 = 6 * np.exp(-3.5 * np.abs(-S + .05))
        V = (V + V2) / 2
        
    return 1 / V



# From Rademaker et al. (2012)
# weight of traveler is given in kg
# weight of pack is given in kg
# terrain coefficients greater than 1 introduce "friction"
# velocity is Walking speed in meters per second
# S is supplied in DEGREES
def cost_rademaker(S,weight=50,pack_weight=0,terrain_coefficient=1.1,velocity=1.2):
   
    # Rademaker assumes a grade in percent (0 to 100, rather than 0 to 1):
    G = 100 * np.arctan(np.deg2rad(S))
    
    W = weight
    L = pack_weight
    tc = terrain_coefficient
    V = velocity
    
    # Cost, in MWatts
    MW = 1.5*W + 2.0 * (W + L) * ((L/W)**2) + tc * (W+L) * (1.5 * V**2 + .35 * V * G)
    
    return MW


#S is in DEGREES.  scale_factor corresponds to slope tolerance, approximately related to the mean
#typical slope.  A higher number will go through rougher terrain.
def cost_pingel_exponential(S,scale_factor=9.25):
    EXP = stats.expon.pdf(0,0,scale_factor) / stats.expon.pdf(S,0,scale_factor) 
    return EXP

# Vertical exaggeration.  S is in DEGREES.
def ve(S,ve=2.3):
    S = np.tan(np.deg2rad(S))
    S = np.rad2deg(np.arctan(ve *  S))
    return S

# Changes from nodes/edges to list of starts and list of lists of ends matching the starts.
def get_lists(nodes,edges):
    start_list = edges.iloc[:,0].unique()
    end_list = [edges.iloc[:,1].loc[edges.iloc[:,0]==item].values for item in start_list] 
    
    start_coords = []
    end_coords = []
    ids = []
    
    for i, this_start in enumerate(start_list):
        these_ends = end_list[i]
        
        these_ids = [this_start + '_to_' + te for te in these_ends]
        these_start_coords = nodes.loc[this_start,'coords']
        these_end_coords = nodes.loc[these_ends,'coords'].values
        
        start_coords.append(these_start_coords)
        end_coords.append(these_end_coords)
        ids.append(these_ids)


    return start_list, end_list, ids, start_coords, end_coords


# Given start and ends, make a geodataframe of straight lines connecting them.
def direct_routes(nodes,edges):
    
    start_list, end_list, ids, start_coords, end_coords = get_lists(nodes,edges)

    gdf = pd.DataFrame()
    
    for i,this_start in enumerate(start_coords):
        df = pd.DataFrame()
        these_end_coords = end_coords[i]
        df['ids'] = ids[i]
        df['method'] = 'direct'
        df['geometry'] = [LineString([this_start,this_end]) for this_end in these_end_coords]
        
        gdf = gdf.append(df,ignore_index=True)

    gdf = geopandas.GeoDataFrame(gdf,geometry=gdf['geometry'],crs=4326)
    
    return gdf


# A helper function to translate lat/lon to map coords to pixel coords.
def lcp_coordinate_conversion(start_coords,end_coords,crs):
    
    converted_start_coords = []
    converted_end_coords = []
    
    for i,this_start_coord in enumerate(start_coords):
        these_end_coords = end_coords[i]
        
        # Convert from lat/lon to map coordinates
        this_start_coord = neilpy.coord_transform(*this_start_coord,4326,crs)
        these_end_coords = [neilpy.coord_transform(*item,4326,crs) for item in these_end_coords]
        
        # Convert from map coordinates to pixel coordinates
        this_start_coord = (~meta['transform']*this_start_coord)[::-1]
        these_end_coords = [(~meta['transform']*item)[::-1] for item in these_end_coords]
        
        # Round them to ints
        this_start_coord = tuple(np.round(this_start_coord).astype(np.uint32))
        these_end_coords = [tuple(item) for item in np.round(these_end_coords).astype(np.uint32)]
        
        converted_start_coords.append(this_start_coord)
        converted_end_coords.append(these_end_coords)
        
    return converted_start_coords, converted_end_coords



# Use skimage.MCP to calculate routes, return a geodataframe
def get_areal_routes(nodes,edges,surface,meta,label='areal'):
    
    gdf = pd.DataFrame()

    print('Creating surface network for',label)
    m = MCP_Geometric(surface,fully_connected=True)
    print('Done creating surface network.')
    
    start_list, end_list, ids, start_coords, end_coords = get_lists(nodes,edges)
    
    conv_start_coords, conv_end_coords = lcp_coordinate_conversion(start_coords,end_coords,meta['crs'])
    
    for i,this_start_coord in enumerate(conv_start_coords):
        these_end_coords = conv_end_coords[i]

        print('Calculating costs and routes.')
        costs, traceback_array  = m.find_costs([this_start_coord],these_end_coords,find_all_ends=True)
        print('Done calculating costs and routes.')
        
        # Pull routes and convert
        routes = [m.traceback(this_end_coord) for this_end_coord in these_end_coords]
        geometries= [LineString(np.vstack(meta['transform']*np.fliplr(route).T).T) for route in routes]
        
        df = pd.DataFrame()
        df['ids'] = ids[i]
        df['method'] = label
        df['geometry'] = geometries
        gdf = gdf.append(df,ignore_index=True)
        
        gdf = geopandas.GeoDataFrame(gdf,geometry=gdf['geometry'],crs=meta['crs'])
        
    
    return gdf


In [19]:
# Load data

# nodes must have a unique ID in the first column, and then longitudes and latitudes in columns 2 and 3.
# edges must have start and destination unique IDs that match those in nodes

nodes = pd.read_csv('data/nodes.csv',index_col=0)
edges = pd.read_csv('data/edges.csv')
nodes['coords'] = list(zip(nodes.iloc[:,0], nodes.iloc[:,1]))    

E, meta = neilpy.imread('data/cusco.tif')
S = neilpy.slope(E,meta['cellsize'])

In [20]:
# Calculate direct routes.  Note, "out" directory MUST exist.

gdf = direct_routes(nodes,edges).to_crs(meta['crs'])
gdf.to_file('out/direct.shp')

In [21]:
# Calculate a variety of LCPs to demonstrate the capability

gdf = get_areal_routes(nodes,edges,S,meta,label='slope')
gdf.to_file('out/areal.shp')

C = cost_tobler_hiking_function(S)
gdf = get_areal_routes(nodes,edges,C,meta,label='tobler')
gdf.to_file('out/tobler.shp')

label = 'tobler_ve'
C = cost_tobler_hiking_function(ve(S))
gdf = get_areal_routes(nodes,edges,C,meta,label=label)
gdf.to_file('out/' + label + '.shp')

label = 'pingel_9.25'
C = cost_pingel_exponential(S,scale_factor=9.25)
gdf = get_areal_routes(nodes,edges,C,meta,label=label)
gdf.to_file('out/' + label + '.shp')

label = 'pingel_6'
C = cost_pingel_exponential(S,scale_factor=6)
gdf = get_areal_routes(nodes,edges,C,meta,label=label)
gdf.to_file('out/' + label + '.shp')

label = 'rademaker'
C = cost_rademaker(S)
gdf = get_areal_routes(nodes,edges,C,meta,label=label)
gdf.to_file('out/' + label + '.shp')

Creating surface network for slope
Done creating surface network.
Calculating costs and routes.
Done calculating costs and routes.
Calculating costs and routes.
Done calculating costs and routes.
Creating surface network for tobler
Done creating surface network.
Calculating costs and routes.
Done calculating costs and routes.
Calculating costs and routes.
Done calculating costs and routes.
Creating surface network for tobler_ve
Done creating surface network.
Calculating costs and routes.
Done calculating costs and routes.
Calculating costs and routes.
Done calculating costs and routes.
Creating surface network for pingel_9.25
Done creating surface network.
Calculating costs and routes.
Done calculating costs and routes.
Calculating costs and routes.
Done calculating costs and routes.
Creating surface network for pingel_6
Done creating surface network.
Calculating costs and routes.
Done calculating costs and routes.
Calculating costs and routes.
Done calculating costs and routes.
Creati